# Stock Portfolio Optimization

Consider we have 3 stocks, we expect these stocks to take a certain future value at a given variance. Our task is to spend all our allocated budget to buy the stocks so that we can cross the expected profit levels.

In [13]:
# installing pyomo
!pip install pyomo


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [3]:
from APMonitor import *

In [5]:
from apm import *

In [15]:
dir(pyomo)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'common',
 'contrib',
 'core',
 'dae',
 'dataportal',
 'duality',
 'environ',
 'gdp',
 'mpec',
 'neos',
 'network',
 'opt',
 'repn',
 'scripting',
 'solvers',
 'util',
 'version']

In [16]:
import pandas as pd

In [17]:
# read sample excel sheet
data = pd.read_excel("C:/Users/User/OneDrive/Documents/OR PROJECTS/Stock_Sample_Data_1.xlsx")

In [18]:
data

,Stock,Current Price,Expected Future Price,Variance
0,1,50,55,100
1,2,40,50,1600
2,3,25,20,100


In [47]:
# taking lists from the dataset

stock = data.index
curr_price = data['Current Price']
exp_price = data['Expected Future Price']
variance = data['Variance']

Budget = 100000
expected_return = 110000
exp_return_2 = 200000

In [48]:
# Creating a Pyomo concrete model
model_1 = ConcreteModel()

In [49]:
# creating variables
model_1.stks = Set(initialize = stock)
model_1.x = Var(model_1.stks, domain= NonNegativeReals)

In [50]:
# defining objective function
model_1.obj = Objective(expr = sum(variance[stks]**2 * model_1.x[stks] for stks in model_1.stks), sense =minimize)

In [51]:
# defining constraints
model_1.budget_constraints = Constraint(expr = sum(model_1.x[stks]*curr_price[stks] for stks in model_1.stks) <=Budget)
model_1.budget_constraints = Constraint(expr = sum(model_1.x[stks]*exp_price[stks] for stks in model_1.stks) >= exp_return_2)

(type=<class 'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [52]:
solver = SolverFactory('ipopt', executable='C:/Users/User/AppData/Local/Programs/Python/Python312/Scripts/Ipopt/bin/ipopt.exe', tee=True)
solver.solve(model_1)

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 1, 'Number of variables': 3, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.14.13\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.08388066291809082}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [53]:
# displayig the results
print("Objective Value :", value(model_1.obj))
print("Variable Values:")
for stk in model_1.stks:
    print(f"x_{stk}:", value(model_1.x[stk]))

Objective Value : 36363636.320074275
Variable Values:
x_0: 3636.3636345609084
x_1: -9.974851659129978e-09
x_2: 8.089787738791755e-11
